# 각 사건별 유의미성 검정

In [14]:
import pandas as pd
import numpy as np
from scipy import stats

# 결과 저장용
final_results = []
final_skips = []

# 사건 데이터 로드
incidents_df = pd.read_csv("../데이터/스캔들/ALL논란_Final.csv", parse_dates=['사건 날짜'])

# 벤치마크 지수 로드 및 수익률 계산
kosdaq_df = pd.read_csv("../데이터/주가/코스닥.csv", parse_dates=['날짜'])
kosdaq_df = kosdaq_df.sort_values('날짜').set_index('날짜')
kosdaq_df['return'] = np.log(kosdaq_df['종가'] / kosdaq_df['종가'].shift(1))

# 분석 시작
for idx, row in incidents_df.iterrows():
    agency = row["소속사"]
    incident_date = row["사건 날짜"]
    incident_name = row["사건 내용"]
    group = row["그룹"]
    celebrity = row["연예인 이름"]

    # 소속사 주가 데이터 로드
    stock_df = pd.read_csv(f"../데이터/주가/{agency}.csv", parse_dates=['날짜'])

    stock_df = stock_df.sort_values('날짜').set_index('날짜')
    stock_df['종가'] = pd.to_numeric(stock_df['종가'].astype(str).str.replace(',', ''), errors='coerce')
    stock_df['return'] = np.log(stock_df['종가'] / stock_df['종가'].shift(1))

    # 사건일 보정: 소속사와 코스닥에 모두 존재하도록 가장 가까운 미래 영업일로 보정
    future_stock_dates = stock_df.index[stock_df.index >= incident_date]
    future_benchmark_dates = kosdaq_df.index[kosdaq_df.index >= incident_date]
    possible_dates = future_stock_dates.intersection(future_benchmark_dates)
    
    if len(possible_dates) == 0:
        final_skips.append({
            "사유": "사건일 보정 실패",
            "소속사": agency,
            "사건내용": incident_name
        })
        continue
    incident_date_adj = possible_dates[0]

    
    # 공통 영업일 확보 (사건일 +1,+2일 후)
    common_dates = stock_df.index.intersection(kosdaq_df.index).sort_values()
    if incident_date_adj not in common_dates:
        future_common = common_dates[common_dates > incident_date_adj]
        if len(future_common) == 0:
            final_skips.append({
                "사유": "공통일 보정 실패",         #사건 발생 이후의 데이터 없는 경우.
                "소속사": agency,
                "사건내용": incident_name
            })
            continue
        incident_date_adj = future_common[0]
    
    center_idx = list(common_dates).index(incident_date_adj)
    if center_idx + 3 >= len(common_dates):
        final_skips.append({
            "사유": "이벤트 윈도우 공통일 부족",    #사건 발생 이후의 데이터 부족
            "소속사": agency,
            "사건내용": incident_name
        })
        continue
    window_dates = common_dates[center_idx+1 : center_idx+3]
    
    # 이벤트 AR 및 CAR
    event_returns = stock_df.loc[window_dates]["return"]
    event_benchmark = kosdaq_df.loc[window_dates]["return"]
    event_ar = event_returns.values - event_benchmark.values
    event_car = np.sum(event_ar)

    # 정상 구간 설정   
    prev_dates = stock_df.index[stock_df.index < incident_date_adj]
    if len(prev_dates) == 0:
        final_skips.append({
            "사유": "정상 구간 없음",
            "소속사": agency,
            "사건내용": incident_name
        })
        continue

    incident_prev_day = prev_dates[-1]
    normal_start_date = stock_df.index.min()  # 소속사별 주가 데이터 시작일
    normal_returns = stock_df.loc[normal_start_date:incident_prev_day]["return"]
    normal_benchmark = kosdaq_df["return"].reindex(normal_returns.index)
    valid_idx = normal_returns.notna() & normal_benchmark.notna()
    normal_ar = normal_returns[valid_idx] - normal_benchmark[valid_idx]

        
    # 단일표본 t-검정
    t_stat, p_value = stats.ttest_1samp(normal_ar, event_car)

    final_results.append({
        "소속사": agency,
        "그룹": group,
        "연예인 이름": celebrity,
        "사건 내용": incident_name,
        "사건 날짜" : incident_date,
        "매핑된 주가 날짜": incident_date_adj,
        "CAR": event_car,
        "t통계량": t_stat,
        "p값": p_value
    })

# 데이터 프레임으로 변환
results_df = pd.DataFrame(final_results)
skips_df = pd.DataFrame(final_skips)

In [15]:
len(results_df)

363

In [16]:
len(results_df[results_df['p값']<0.05])

343

유의미한 사건들 저장

In [17]:
last=results_df[results_df['p값']<0.05]
last.to_csv('../데이터/사건별_정당성_검정결과_tmp.csv', index=False, encoding='utf-8-sig')

지은 추가

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

abnormal = pd.read_csv("Final_라벨결측제거_원핫인코딩.csv")
# 사건별 CAR 계산 및 단일표본 t-검정
car_results = []

grouped = abnormal.groupby(['소속사', '연예인 이름', '사건 날짜(매핑된 주가 날짜)'])

for key, group in grouped:
    ar_values = group['비정상 수익률'].dropna().values
    # if len(ar_values) < 2:
    #     continue  # 데이터 부족 시 건너뜀

    car = ar_values.sum()
    t_stat, p_val = stats.ttest_1samp(ar_values, 0)

    metadata = group.iloc[0]  # 동일 사건이므로 첫 행에서 정보 추출

    car_results.append({
        '소속사': key[0],
        '연예인 이름': key[1],
        '사건 날짜(매핑된 주가 날짜)': key[2],
        '사건 날짜': metadata['사건 날짜'],
        '그룹': metadata['그룹'],
        '사건 내용': metadata['사건 내용'],
        'CAR': car,
        't통계량': t_stat,
        'p값': p_val,
        '라벨 1': metadata['라벨 1'],
        '라벨 2': metadata['라벨 2'],
        '라벨 3': metadata['라벨 3']
    })

car_df = pd.DataFrame(car_results)


KeyError: '비정상 수익률'